# Building a Content Based Movies Recommendation System
Content-based filtering uses item features to recommend other items similar to what the user likes, based on their previous actions or explicit feedback.

In [2]:
## Importing Librabries
import ast
import numpy as np
import pandas as pd

#### There are a lots of Datasets let's read one by one

In [3]:
credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [5]:
Id = keywords['id']

In [6]:
movies = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
movies.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


##### We are going to ignore these files :-
1. ratings       == This is a subset of 100,000 ratings from 700 users on 9,000 movies which is already included as a                               average_vote in movies dataset.
2. links         == The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset which                     is also not useful to us in this project.
3. links_small  == Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset again this is also not                      usefull to us in this project.
4. ratings_small   == The subset of 100,000 ratings from 700 users on 9,000 movies which is not usefull to us.

In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [8]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [9]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [10]:
## Let's add those three files movies, credits and keywords based on id
df = credits.merge(keywords,on='id')

In [11]:
df.head()

,cast,crew,id,keywords
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [12]:
data = pd.concat([df,movies], axis=1)

# Let's Jump into it with our dataset

In [13]:
data.head(2)

,cast,crew,id,keywords,adult,belongs_to_collection,budget,genres,homepage,id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46496 entries, 0 to 46495
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cast                   46496 non-null  object 
 1   crew                   46496 non-null  object 
 2   id                     46496 non-null  int64  
 3   keywords               46496 non-null  object 
 4   adult                  45466 non-null  object 
 5   belongs_to_collection  4494 non-null   object 
 6   budget                 45466 non-null  object 
 7   genres                 45466 non-null  object 
 8   homepage               7782 non-null   object 
 9   id                     45466 non-null  object 
 10  imdb_id                45449 non-null  object 
 11  original_language      45455 non-null  object 
 12  original_title         45466 non-null  object 
 13  overview               44512 non-null  object 
 14  popularity             45461 non-null  object 
 15  po

In [15]:
data.isnull().sum()

cast                         0
crew                         0
id                           0
keywords                     0
adult                     1030
belongs_to_collection    42002
budget                    1030
genres                    1030
homepage                 38714
id                        1030
imdb_id                   1047
original_language         1041
original_title            1030
overview                  1984
popularity                1035
poster_path               1416
production_companies      1033
production_countries      1033
release_date              1117
revenue                   1036
runtime                   1293
spoken_languages          1036
status                    1117
tagline                  26084
title                     1036
video                     1036
vote_average              1036
vote_count                1036
dtype: int64

In [16]:
## Let's drop the null values
data = data.drop(['belongs_to_collection','homepage','tagline'],axis=1)
data = data.dropna()

In [17]:
data.shape

(44048, 25)

#### Now, We are going to drop some columns and include some important columns. Let's consider these columns:--
1. cast
2. id
3. keywords
4. adult
5. genres
6. title
7. production company
8. production countary
9. original languague
10. overview
11. popularity
12. budget
13. crew

###### Rest we are not considering because according to me they are not affect in thay way in content based filtering.

In [18]:
movies = data[['adult','cast','crew','keywords','genres','title','production_companies','production_countries','original_language','overview','popularity','budget']]
movies = pd.concat([movies,Id],axis=1)

In [19]:
movies.head()

,adult,cast,crew,keywords,genres,title,production_companies,production_countries,original_language,overview,popularity,budget,id
0,False,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,30000000,862
1,False,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",en,When siblings Judy and Peter discover an encha...,17.015539,65000000,8844
2,False,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",en,A family wedding reignites the ancient feud be...,11.7129,0,15602
3,False,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",en,"Cheated on, mistreated and stepped on, the wom...",3.859495,16000000,31357
4,False,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",en,Just when George Banks has recovered from his ...,8.387519,0,11862


In [20]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46419 entries, 0 to 46418
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   adult                 44048 non-null  object
 1   cast                  44048 non-null  object
 2   crew                  44048 non-null  object
 3   keywords              44048 non-null  object
 4   genres                44048 non-null  object
 5   title                 44048 non-null  object
 6   production_companies  44048 non-null  object
 7   production_countries  44048 non-null  object
 8   original_language     44048 non-null  object
 9   overview              44048 non-null  object
 10  popularity            44048 non-null  object
 11  budget                44048 non-null  object
 12  id                    46419 non-null  int64 
dtypes: int64(1), object(12)
memory usage: 5.0+ MB


In [21]:
movies.isnull().sum()

adult                   2371
cast                    2371
crew                    2371
keywords                2371
genres                  2371
title                   2371
production_companies    2371
production_countries    2371
original_language       2371
overview                2371
popularity              2371
budget                  2371
id                         0
dtype: int64

In [22]:
movies = movies.dropna()

In [23]:
movies.shape

(44048, 13)

# Now, We have prepared the dataset and we are going to do data cleaing

In [24]:
movies.head(1)

,adult,cast,crew,keywords,genres,title,production_companies,production_countries,original_language,overview,popularity,budget,id
0,False,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,30000000,862


In [25]:
movies.duplicated().sum()

0

In [26]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44048 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   adult                 44048 non-null  object
 1   cast                  44048 non-null  object
 2   crew                  44048 non-null  object
 3   keywords              44048 non-null  object
 4   genres                44048 non-null  object
 5   title                 44048 non-null  object
 6   production_companies  44048 non-null  object
 7   production_countries  44048 non-null  object
 8   original_language     44048 non-null  object
 9   overview              44048 non-null  object
 10  popularity            44048 non-null  object
 11  budget                44048 non-null  object
 12  id                    44048 non-null  int64 
dtypes: int64(1), object(12)
memory usage: 4.7+ MB


In [27]:
## Function for cleaning the genres and keywords columns.

def convert(object):
    lst = []
    for i in ast.literal_eval(object):
        lst.append(i['name'])
    
    return lst       

In [28]:
## Function for cleaning the cast,production_companies and production_countries columns
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [29]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['production_companies'] = movies['production_companies'].apply(convert3)
movies['production_countries'] = movies['production_countries'].apply(convert3)


movies.head(2)

,adult,cast,crew,keywords,genres,title,production_companies,production_countries,original_language,overview,popularity,budget,id
0,False,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",Toy Story,[Pixar Animation Studios],[United States of America],en,"Led by Woody, Andy's toys live happily in his ...",21.946943,30000000,862
1,False,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]",Jumanji,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],en,When siblings Judy and Peter discover an encha...,17.015539,65000000,8844


In [30]:
## Let's remove spaces from the columns

def remove(L):
    L1=[]
    for i in L:
        L1.append(i.replace(" ",""))
    return L1    

In [31]:
movies['cast'] = movies['cast'].apply(remove)
movies['genres'] = movies['genres'].apply(remove)
movies['keywords'] = movies['keywords'].apply(remove)
movies['production_companies'] = movies['production_companies'].apply(remove)
movies['production_countries'] = movies['production_countries'].apply(remove)

movies.head(2)

,adult,cast,crew,keywords,genres,title,production_companies,production_countries,original_language,overview,popularity,budget,id
0,False,"[TomHanks, TimAllen, DonRickles, JimVarney, Wa...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",Toy Story,[PixarAnimationStudios],[UnitedStatesofAmerica],en,"Led by Woody, Andy's toys live happily in his ...",21.946943,30000000,862
1,False,"[RobinWilliams, JonathanHyde, KirstenDunst, Br...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgame, disappearance, basedonchildren'sbo...","[Adventure, Fantasy, Family]",Jumanji,"[TriStarPictures, TeitlerFilm, InterscopeCommu...",[UnitedStatesofAmerica],en,When siblings Judy and Peter discover an encha...,17.015539,65000000,8844


In [32]:
## Let's define a function for extracting Director name
def director(object):
    L=[]
    for i in ast.literal_eval(object):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L        

In [33]:
movies['crew'] = movies['crew'].apply(director)

In [34]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['original_language'] = movies['original_language'].apply(lambda x:x.split())

In [35]:
movies['tags'] = movies['overview'] + movies['cast'] +movies['crew'] + movies['keywords'] + movies['genres'] + movies['production_companies'] + movies['production_countries'] + movies['original_language']

In [36]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew','production_companies','production_countries','original_language'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()


,adult,title,popularity,budget,id,tags
0,False,Toy Story,21.946943,30000000,862,"Led by Woody, Andy's toys live happily in his ..."
1,False,Jumanji,17.015539,65000000,8844,When siblings Judy and Peter discover an encha...
2,False,Grumpier Old Men,11.7129,0,15602,A family wedding reignites the ancient feud be...
3,False,Waiting to Exhale,3.859495,16000000,31357,"Cheated on, mistreated and stepped on, the wom..."
4,False,Father of the Bride Part II,8.387519,0,11862,Just when George Banks has recovered from his ...


In [37]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44048 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   adult       44048 non-null  object
 1   title       44048 non-null  object
 2   popularity  44048 non-null  object
 3   budget      44048 non-null  object
 4   id          44048 non-null  int64 
 5   tags        44048 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


In [38]:
import re ## Regular Expressions
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [39]:
def stemm(text):
    L =[]
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)    

In [40]:
new['tags'] = new['tags'].apply(stemm)

In [41]:
new['tags'][1]

"when sibl judi and peter discov an enchant board game that open the door to a magic world, they unwittingli invit alan -- an adult who' been trap insid the game for 26 year -- into their live room. alan' onli hope for freedom is to finish the game, which prove riski as all three find themselv run from giant rhinoceroses, evil monkey and other terrifi creatures. robinwilliam jonathanhyd kirstendunst bradleypierc bonniehunt bebeneuwirth davidalangri patriciaclarkson adamhann-byrd laurabellbundi jameshandi gillianbarb brandonobray cyrusthiedek garyjosephthorup leonardzola lloydberri malcolmstewart annabelkershaw darrylhenriqu robyndriscol peterbry sarahgilson floricavlad junelion brendalockmul joe johnston boardgam disappear basedonchildren'sbook newhom reclus giantinsect adventur fantasi famili tristarpictur teitlerfilm interscopecommun unitedstatesofamerica en"

In [42]:
new['tags'][2]

"a famili wed reignit the ancient feud between next-door neighbor and fish buddi john and max. meanwhile, a sultri italian divorcé open a restaur at the local bait shop, alarm the local who worri she'll scare the fish away. but she' less interest in seafood than she is in cook up a hot time with max. waltermatthau jacklemmon ann-margret sophialoren darylhannah burgessmeredith kevinpollak howard deutch fish bestfriend duringcreditssting oldmen romanc comedi warnerbros. lancasterg unitedstatesofamerica en"

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=5000,stop_words='english')

In [44]:
vectors = tv.fit_transform(new['tags']).toarray()

In [45]:
vectors.shape

(44048, 5000)

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
similarity = cosine_similarity(vectors)

In [48]:
similarity

array([[0.20452931, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0136759 , 0.        , ..., 0.        , 0.        ,
        0.01071441],
       [0.        , 0.        , 0.09336596, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.00135653,
        0.00300057],
       [0.        , 0.        , 0.        , ..., 0.00135653, 1.        ,
        0.00133734],
       [0.        , 0.01071441, 0.        , ..., 0.00300057, 0.00133734,
        1.        ]])

In [49]:
def recommend(movie):
    index = new[new['title']==movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [50]:
recommend('Dangal')

Motorcycle Gang
Dave Chappelle: The Age of Spin
Perfect Opposites
The Last Patrol
Highway Patrolman


In [51]:
import pickle

In [ ]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))